# With rusterize

Rasterix includes a dask-aware wrapper for [rusterize](https://github.com/ttrotto/rusterize), a fast Rust-based rasterization library.

```{tip}
Unlike the `rasterio` engine, `rusterize` does **not** require GDAL! This makes it much easier to install and is a great choice when GDAL installation is problematic.
```

```{note}
The `rusterize` engine does not support the `all_touched` parameter. Use `engine='rasterio'` if you need `all_touched` support.
```

## Read data

### Read in some raster data

In [ ]:
import xarray as xr
import xproj  # noqa

ds = xr.tutorial.open_dataset("eraint_uvz")
ds = ds.proj.assign_crs(spatial_ref="epsg:4326")
ds

### Read in example geometries

In [ ]:
import geodatasets
import geopandas as gpd

world = gpd.read_file(geodatasets.get_path("naturalearth land"))
world

In [ ]:
from rasterix.rasterize import rasterize

n = rasterize(ds, world[["geometry"]], xdim="longitude", ydim="latitude", engine="rusterize")
n.plot()

## Out-of-core support

All combinations of chunked and in-memory arrays and geometries are supported.

### dask.array + geopandas

In [ ]:
chunked = ds.chunk({"latitude": -1, "longitude": 120})
d = rasterize(chunked, world[["geometry"]], xdim="longitude", ydim="latitude", engine="rusterize")
d

In [ ]:
xr.testing.assert_identical(n, d)

### dask.array + dask-geopandas

In [ ]:
import dask_geopandas as dgpd

dd = rasterize(
    ds.chunk({"latitude": -1, "longitude": 240}),
    dgpd.from_geopandas(world[["geometry"]], npartitions=3),
    xdim="longitude",
    ydim="latitude",
    engine="rusterize",
)
dd

In [ ]:
xr.testing.assert_identical(dd, d)